# Direct Hashing and Purning (DHP)

### An Effective Hash-Based Algorithm for Mining Association Rules

DHP algorithm is a hash based techniques to improve the performance of Apriori algorithm.DHP algorithm uses a hash function for candidate item set generation and also use pruning to successively reduce the size of transaction database.

> * _Jong Soo Park_
> * _Ming-Syan Chen_
> * _Philip S. Yu_


#### Steps

    1. Scan the database to count the support of candidate-1(C1) item set and select the items who have support     count>=min_sup to add into large item set(L1).
    
    2. Now make possible set of candidate-2 item set in each transaction of database (D2). Hash function is       applied on each candidate-2 item set to find the corresponding bucket number.

    3. Scan database (D2) and hash each item set of transactions into corresponding hash bucket. Some item         sets are hashed into same bucket this is called collision problem.

    4. Select only that candidate-2 item setwhose corresponding bucket count>=min_sup.If there is no collision     then adds into L2.
    
        a. If there is no collision then add the selected item sets into L2.
        
        b. Else one more scan of database is required to count the support of collided item sets and the item set  having support >=min_sup is added into L2.
        
    5. Now make possible set of candidate-3 item set (D3) and repeat the same procedure until Ck=’


In [1]:
import pandas as pd
from itertools import *
import numpy as np

In [2]:
dta =  pd.DataFrame({
    "TID": ["T{}".format(i+1) for i in range(5)],
    "List of Items":[['A','C','D'],['B','C','E'],['A','B','C','E'],['B','E'],['A','B','C','D']]
})

#### Database TDB

In [3]:
dta

,TID,List of Items
0,T1,"[A, C, D]"
1,T2,"[B, C, E]"
2,T3,"[A, B, C, E]"
3,T4,"[B, E]"
4,T5,"[A, B, C, D]"


#### Mínimo soprte

In [4]:
minsup = 2

#### Candidatos principales

In [5]:
#Conseguimos los candidatos principales del data Frame
candidates = set()
for items in dta["List of Items"]:
    candidates = candidates.union(set(items))
print("Candidates = {}".format(candidates))

Candidates = {'B', 'D', 'A', 'E', 'C'}


#### Procedure

In [6]:
#Ésta función realcia el hasheo del elemento
def hashing(itemset,mod,grade,lexicOrder):
    h = 0
    g = grade
    itemset=list(itemset)
    itemset.sort()
    for i in itemset:
        index = lexicOrder.index(i)
        h += (index+1)*g
        g = g//10
    return h % mod

#### Implementación DHP

In [12]:
def DHP(candidates,minsup,dataFrame,sizeHasTab): 
    #Oredenamos lexicograficamente los elementos
    LexicOrder = list(candidates)
    LexicOrder.sort()
    mod = sizeHasTab + 1 #Módulo
    C = candidates #Conjunto de candidatos
    Di = dataFrame
    tuplas = list() #Lista pra formar un Data Frame
    x =set()
    #Hallamos L1 a partir de los candidatos
    for c in C:
        supp = 0
        A = set(c)
        for t in dataFrame["List of Items"]:
            B = set(t)
            if A.issubset(B):
                supp +=1
        if supp >= minsup:
            x = x.union(A)
            tuplas.append([A,supp])
    L1 = pd.DataFrame(data=np.array(tuplas),columns=["ItemSet","Support"])
    print("\n {} \n L1 :\n\n".format("-"*40),L1)
    
    k = 2
    Ck = set(combinations(x,k))
    while Ck: #Mientras el conjunto Ck no esté vacio        
        #Creamos una nueva tabla D cuyos items set seran de grupos de tamaño k
        w = list()
        for iSet in dataFrame["List of Items"]:
            A = set()            
            for e in iSet:
                B = set(e)
                A = A.union(B)        
            if A.issubset(C) and len(A)>=k:
                s = set(combinations(A,k))
                w.append(s)
                
        Di = pd.DataFrame(data=np.array(w),columns=["List of Items"])
        print("\n {} \n D{} \n\n".format("-"*40,k),Di)

        #Creamos una tabla hash de tamaño sizeHashTab
        hashTable = [[] for i in range(sizeHasTab+1)]

        #Hasheamos cada itemset de la tabla nueva y lo colocamos en un buket STEP 3
        grade = 10**(k-1)
        for iSet in Di["List of Items"]:        
            for e in iSet:            
                buket = hashing(e,mod,grade,LexicOrder)
                hashTable[buket].append(set(e)) #Colocamos en el buket el elemento e

        print("\n {} \n HASH TABLE : \n\n".format("-"*40),hashTable)       

        #Generamos Li X Li
        lxl = set(combinations(set(x),k))
        
        #Generamos L(i+1) dados los candidtos lxl STEP 4
        C = set()
        tuplas = list()
        x = set()
        for e in lxl:
            index_buket = hashing(e,mod,grade,LexicOrder)
            if len(hashTable[index_buket]) >= minsup:            
                A = set(e)
                C = C.union(A)                
                supp =  hashTable[index_buket].count(A)
                if supp >= minsup:
                    x = x.union(A)
                    tuplas.append(["{} --- {}".format(A,supp)])
        Li = pd.DataFrame(data=np.array(tuplas),columns=["Support"])
        print("\n {} \n L{} :\n\n".format("-"*40,k),Li)
        k +=1
        Ck = set(combinations(x,k))

In [13]:
c = candidates #Candidatos principales
ms = minsup #Mínimo soporte defnido
dataFrame = dta #Tabla o dataFrame de las trasnsacciones 
n = 6 #Tamaño de la tabla hash 
DHP(c,ms, dataFrame,n)


 ---------------------------------------- 
 L1 :

   ItemSet Support
0     {B}       4
1     {D}       2
2     {A}       3
3     {E}       3
4     {C}       4

 ---------------------------------------- 
 D2 

                                       List of Items
0                          {(D, C), (A, D), (A, C)}
1                          {(B, C), (B, E), (E, C)}
2  {(A, E), (A, C), (B, E), (B, C), (E, C), (A, B)}
3                                          {(B, E)}
4  {(A, C), (B, D), (B, C), (A, D), (D, C), (A, B)}

 ---------------------------------------- 
 HASH TABLE : 

 [[{'A', 'D'}, {'E', 'C'}, {'E', 'C'}, {'A', 'D'}], [{'A', 'E'}], [{'B', 'C'}, {'B', 'C'}, {'B', 'C'}], [{'B', 'D'}], [{'B', 'E'}, {'B', 'E'}, {'B', 'E'}], [{'A', 'B'}, {'A', 'B'}], [{'D', 'C'}, {'A', 'C'}, {'A', 'C'}, {'A', 'C'}, {'D', 'C'}]]

 ---------------------------------------- 
 L2 :

             Support
0  {'A', 'B'} --- 2
1  {'A', 'C'} --- 3
2  {'D', 'C'} --- 2
3  {'B', 'C'} --- 3
4  {'B', 'E'} --- 3
5